# Calculate profit of each transaction

Import packages and data.

In [1]:
import csv
import requests
import pandas as pd
import datetime as dt
from decimal import Decimal
from decimal import getcontext
import plotly.express as px
getcontext().prec = 30

In [2]:
uniswap_history  = pd.read_pickle('./uniswap_events.pickle')

## First method to calculate profit

In [3]:
def transaction_unit_price(row):
    if(row.event == 'TokenPurchase'):
#         row = uniswap_history.iloc[row.name-1] 
        spot_price = row.eth_delta / row.token_delta # corrigir para deltas e nao balanços
        return Decimal(spot_price)
    elif(row.event == 'EthPurchase'):
#         row = uniswap_history.iloc[row.name-1] 
        spot_price = row.token_delta / row.eth_delta
        return Decimal(spot_price)
    
def uniswap_price_after_transaction(row):
    if(row.event == 'TokenPurchase'):
        spot_price = row.eth_balance / row.token_balance
        return Decimal(spot_price)
    elif(row.event == 'EthPurchase'):
        spot_price = row.token_balance / row.eth_balance
        return Decimal(spot_price)
    
def calc_profit(row):
    if(row.event == 'EthPurchase'):
        return int(row.gain_price*abs(row.eth_delta))
    elif(row.event == 'TokenPurchase'):
        return int(row.gain_price*(abs(row.token_delta*row.model_price)))
    
def convert_to_rfc(block_timestamp):
    date = block_timestamp.date().isoformat()+"T00%3A00%3A00Z"
    return date 

def classifier(row, c_rule):
    if(row.event == 'EthPurchase'):
        delta_I = row.eth_delta
        delta_O = row.token_delta
    elif(row.event == 'TokenPurchase'):
        delta_I = row.token_delta
        delta_O = row.eth_delta
    
    if (delta_I / (10 ** (18-c_rule))).is_integer() or (delta_O / (10 ** (18-c_rule))).is_integer():
        return "Conv"
    else:
        return "Arb"

def classify_transaction(start, end, gain_price):
    key="380cc9d88b57b2771232f2b3eb5442ea"
    coin="ETH"
    api_request = f"https://api.nomics.com/v1/exchange-rates/history?key={key}&currency={coin}&start={start}&end={end}"
    response = requests.get(api_request).json()[0]
    if(not len(response)):
        print(start, end)
    convert_to_ether = int(1/float(response['rate'])*(1*(10**18)))
    return convert_to_ether

In [4]:
first_method = uniswap_history.copy()

In [5]:
first_method['transaction_price'] = first_method.apply(lambda row: transaction_unit_price(row), axis=1)
first_method['model_price'] = first_method.apply(lambda row: uniswap_price_after_transaction(row), axis=1)
first_method['gain_price'] = first_method['model_price'] - first_method['transaction_price']
first_method['profit'] = first_method.apply(lambda row: calc_profit(row), axis=1)
first_method.dropna(inplace=True)
first_method['profit'] = first_method.apply(lambda row: int(row.profit), axis=1)
first_method['block_date'] = first_method.apply(lambda row: convert_to_rfc(row.block_timestamp), axis=1)

In [6]:
start = first_method.iloc[0].block_date
end = first_method.iloc[-1].block_date
key="380cc9d88b57b2771232f2b3eb5442ea"
coin="ETH"
api_request = f"https://api.nomics.com/v1/exchange-rates/history?key={key}&currency={coin}&start={start}&end={end}&format=csv"

with requests.Session() as s:
    download = s.get(api_request)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    rate_df = pd.DataFrame(my_list, columns=['date', 'convert_rate'])

In [7]:
first_method['block_timestamp'] = pd.to_datetime(first_method['block_timestamp']).dt.strftime('%Y-%m-%d')
rate_df['date'] = pd.to_datetime(rate_df['date']).dt.strftime('%Y-%m-%d')

In [8]:
first_method = first_method.set_index('block_timestamp').join(rate_df.set_index('date')).reset_index(drop=True)

In [9]:
first_method

,transactionHash,transaction_index,transaction_sender,address,data,topics,blockNumber_dec,contract,event,contract_event,...,eth_balance,token_balance,UNI_supply,invariant,transaction_price,model_price,gain_price,profit,block_date,rate
0,0xd53f01fc0cabbf4083da93a023ddb81de91320027077...,97,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0x7f4091b46c33e918a0f3aa42307641d17bb67029427...,6629171,DAI,EthPurchase,DAIEthPurchase,...,29974673940746674209,5905000000000000000000,30000000000000000000,177000449620109111204145000000000000000000,-197.4251086593112631817348301410675048828125,196.9996408192090484590153209865093231201171875,394.4247494785203116407501511,9989224576271185920,2018-11-02T00%3A00%3A00Z,201.54637026244962272066371234883213
1,0x53172898dbe437c784f7e4ac757087b28d1b06aa6b72...,53,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0x7f4091b46c33e918a0f3aa42307641d17bb67029427...,6629177,DAI,EthPurchase,DAIEthPurchase,...,29964555511671268196,5907000000000000000000,30000000000000000000,177000629407442181233772000000000000000000,-197.6591410677796147865592502057552337646484375,197.1329091699427635830943472683429718017578125,394.7920502377223783696535975,3994675359864521728,2018-11-02T00%3A00%3A00Z,201.54637026244962272066371234883213
2,0x6cef57fd37d7a2602c1236ff080c61c37e10980c7e3b...,24,0x11e4857bb9993a50c685a79afad4e6f65d518dda,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0xcd60aa75dea3072fbc07ae6d7d856b5dc5f4eee8885...,6629190,DAI,TokenPurchase,DAITokenPurchase,...,29974555511671268196,5905035238623641640532,30000000000000000000,177000806558499339943044863018842396120272,-0.0050896765990660775696996509509517636615782...,0.00507610103926455207695722648963965184520930...,0.01016577763833062964665687744,101386627284592,2018-11-02T00%3A00%3A00Z,201.54637026244962272066371234883213
3,0x13d958222cbec3f6ae93e8f73af0f6ce800541db27a0...,281,0x11e4857bb9993a50c685a79afad4e6f65d518dda,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0xcd60aa75dea3072fbc07ae6d7d856b5dc5f4eee8885...,6629198,DAI,TokenPurchase,DAITokenPurchase,...,29984555511671268196,5903071785798411448958,30000000000000000000,177000983650652913895410492180545782739768,-0.0050930686347545007716663967300974036334082...,0.00507948346212017988465792228680584230460226...,0.01017255209687468065632431902,101454179884857,2018-11-02T00%3A00%3A00Z,201.54637026244962272066371234883213
4,0x51f662378a5e158bece87f19c5537fe69805e7163fe4...,121,0x4defa30195094963cfac7285d8d6e6e523c7f90d,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0xcd60aa75dea3072fbc07ae6d7d856b5dc5f4eee8885...,6629301,DAI,TokenPurchase,DAITokenPurchase,...,29994555511671268196,5901109640216828255225,30000000000000000000,177001160683942120400286097646701713324100,-0.0050964617986863714263479074872975616017356...,0.00508286700983396049630114177375617146026343...,0.01017932880852033192264904926,101521754948534,2018-11-02T00%3A00%3A00Z,201.54637026244962272066371234883213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135604,0x4c4d18cda4948561bf4df0e9cbfac4b7e9aedfea0285...,13,0xb14d1163e4b3cb18edf864b523ed42a87c21f68c,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0xcd60aa75dea3072fbc07ae6d7d856b5dc5f4eee8885...,9193158,DAI,TokenPurchase,DAITokenPurchase,...,5013841555546109396965,661804343765215722063362,3304234867026987131356,3318182120410961321903054254127069667340496330,-0.0078058810846925964430087496737087349174544...,0.00757601790133435436941367058238938625436276...,0.01538189898602695081242242026,8288955528190379,2019-12-31T00%3A00%3A00Z,129.32440406447734391453612660945510
135605,0xf701ddbf48fc7ea18079f5bd9b7a29a5485abbee71ae...,29,0x614800dc2c7372f2595e66ca62cc2fa485f00c20,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0xcd60aa75dea3072fbc07ae6d7d856b5dc5f4eee8885...,9193160,DAI,TokenPurchase,DAITokenPurchase,...,5014336157472525012695,661740995794993307924874,3304234867026987131356,3

In [10]:
first_method['fix_cost'] = first_method.apply(lambda row: int((1/float(row.rate))*(10**18)), axis=1)
first_method['class'] = first_method.apply(lambda row: classifier(row, 5), axis=1)

In [11]:
first_method['valid_transaction'] = first_method['profit'] > first_method['fix_cost']

In [12]:
first_method['valid_transaction']

0          True
1          True
2         False
3         False
4         False
          ...  
135604     True
135605    False
135606     True
135607     True
135608     True
Name: valid_transaction, Length: 135609, dtype: bool

In [13]:
import plotly.express as px

In [16]:
px.histogram(first_method.loc[first_method['class'] == 'Conv']['valid_transaction'])

# arb 66.436k valid and 10.754 non-valid
# conv 47.642 valid and 10.777 non-valid
# total 114.078 valid and 21.531 non-valid

In [32]:
first_method

,transactionHash,transaction_index,transaction_sender,address,data,topics,blockNumber_dec,contract,event,contract_event,...,agent,eth_delta,token_delta,uni_delta,eth_balance,token_balance,UNI_supply,invariant,convert_rate,block_timestamp
0,0xbd773ccc33c831378f2aa2b7d4a9ae8a20ed9112ea9a...,150,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0x06239653922ac7bea6aa2b19dc486b9361821d37712...,6629139,DAI,AddLiquidity,DAIAddLiquidity,...,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,30000000000000000000,5900000000000000000000,0,30000000000000000000,5900000000000000000000,0,177000000000000000000000000000000000000000,201.546370,2018-11-02 10:27:36+00:00
1,0xbd773ccc33c831378f2aa2b7d4a9ae8a20ed9112ea9a...,150,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,6629139,DAI,Transfer,DAITransfer,...,0x0000000000000000000000000000000000000000,0,0,30000000000000000000,30000000000000000000,5900000000000000000000,30000000000000000000,177000000000000000000000000000000000000000,201.546370,2018-11-02 10:27:36+00:00
2,0xd53f01fc0cabbf4083da93a023ddb81de91320027077...,97,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0x7f4091b46c33e918a0f3aa42307641d17bb67029427...,6629171,DAI,EthPurchase,DAIEthPurchase,...,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,-25326059253325791,5000000000000000000,0,29974673940746674209,5905000000000000000000,30000000000000000000,177000449620109111204145000000000000000000,201.546370,2018-11-02 10:33:56+00:00
3,0x53172898dbe437c784f7e4ac757087b28d1b06aa6b72...,53,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0x7f4091b46c33e918a0f3aa42307641d17bb67029427...,6629177,DAI,EthPurchase,DAIEthPurchase,...,0xf369af914dbed0ad7afdddebc631ee0fda1b4891,-10118429075406013,2000000000000000000,0,29964555511671268196,5907000000000000000000,30000000000000000000,177000629407442181233772000000000000000000,201.546370,2018-11-02 10:35:47+00:00
4,0x6cef57fd37d7a2602c1236ff080c61c37e10980c7e3b...,24,0x11e4857bb9993a50c685a79afad4e6f65d518dda,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0xcd60aa75dea3072fbc07ae6d7d856b5dc5f4eee8885...,6629190,DAI,TokenPurchase,DAITokenPurchase,...,0x11e4857bb9993a50c685a79afad4e6f65d518dda,10000000000000000,-1964761376358359468,0,29974555511671268196,5905035238623641640532,30000000000000000000,177000806558499339943044863018842396120272,201.546370,2018-11-02 10:38:18+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147434,0x4c4d18cda4948561bf4df0e9cbfac4b7e9aedfea0285...,13,0xb14d1163e4b3cb18edf864b523ed42a87c21f68c,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0xcd60aa75dea3072fbc07ae6d7d856b5dc5f4eee8885...,9193158,DAI,TokenPurchase,DAITokenPurchase,...,0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,555227276348537093,-71129353666089377834,0,5013841555546109396965,661804343765215722063362,3304234867026987131356,3318182120410961321903054254127069667340496330,129.324404,2019-12-31 23:31:34+00:00
147435,0xf701ddbf48fc7ea18079f5bd9b7a29a5485abbee71ae...,29,0x614800dc2c7372f2595e66ca62cc2fa485f00c20,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0xcd60aa75dea3072fbc07ae6d7d856b5dc5f4eee8885...,9193160,DAI,TokenPurchase,DAITokenPurchase,...,0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,494601926415615730,-63347970222414138488,0,5014336157472525012695,661740995794993307924874,3304234867026987131356,3318191802096709075939561492313859130956275430,129.324404,2019-12-31 23:31:44+00:00
147436,0x06257f58f391b2fa9201cf4fc09e668f1e0b03c59b96...,27,0xffbfdb803d38d794b5785ee0ac09f83b429d11b5,0x09cabec1ead1c0ba254b09efb3ee13841712be14,0x,[0x7f4091b46c33e918a0f3aa42307641d17bb67029427...,9193211,DAI,EthPurchase,DAIEthPurchase,...,0x8018280076d7fa2caa1147e441352e8a89e1ddbe,-2425103627055529511,312610256829251059712,0,501191105384546948318

In [13]:
114.078/(114.078+21.531)

0.8412273521668916

In [29]:
import csv
import requests
import pandas as pd
import datetime as dt

def convert_to_rfc(block_timestamp):
    date = block_timestamp.date().isoformat()+"T00%3A00%3A00Z"
    return date 

uniswap_history  = pd.read_pickle('./uniswap_events.pickle')
first_method = uniswap_history.copy()
first_method.drop(columns=['convert_rate'], inplace=True)

In [30]:
start = convert_to_rfc(first_method.iloc[0].block_timestamp)
end = convert_to_rfc(first_method.iloc[-1].block_timestamp)
key="380cc9d88b57b2771232f2b3eb5442ea"
coin="ETH"
api_request = f"https://api.nomics.com/v1/exchange-rates/history?key={key}&currency={coin}&start={start}&end={end}&format=csv"

with requests.Session() as s:
    download = s.get(api_request)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    rate_df = pd.DataFrame(my_list, columns=['date', 'convert_rate'])
    rate_df['convert_rate'] = rate_df['convert_rate'].astype(float)


first_method['block_timestamp'] = pd.to_datetime(first_method['block_timestamp']).dt.strftime('%Y-%m-%d')
rate_df['date'] = pd.to_datetime(rate_df['date']).dt.strftime('%Y-%m-%d')
first_method = first_method.set_index('block_timestamp').join(rate_df.set_index('date')).reset_index(drop=True)
first_method['block_timestamp'] = uniswap_history['block_timestamp']

In [31]:
first_method.to_pickle('uniswap_events.pickle')

In [25]:
rate_df['convert_rate'].astype(float)

0      201.546370
1      199.980206
2      211.668770
3      210.018836
4      219.794251
          ...    
420    126.682317
421    128.504471
422    134.807236
423    131.742101
424    129.324404
Name: convert_rate, Length: 425, dtype: float64

## Second method

In [27]:
def uniswap_price_before_transaction(row):
    if(row.event == 'TokenPurchase'):
        row = uniswap_history.iloc[row.name-1] 
        spot_price = row.eth_balance / row.token_balance
        return Decimal(spot_price)
    elif(row.event == 'EthPurchase'):
        row = uniswap_history.iloc[row.name-1] 
        spot_price = row.token_balance / row.eth_balance
        return Decimal(spot_price)
    
def uniswap_price_after_transaction(row):
    if(row.event == 'TokenPurchase'):
        spot_price = row.eth_balance / row.token_balance
        return Decimal(spot_price)
    elif(row.event == 'EthPurchase'):
        spot_price = row.token_balance / row.eth_balance
        return Decimal(spot_price)
    
def calc_abs_profit(P, row):
    if(row.event == "TokenPurchase"):
        profit = abs(row.token_delta*P) - row.eth_delta
        return profit
    elif(row.event == "EthPurchase"):
        profit = abs(row.eth_delta) - int(row.token_delta/P)
        return profit
    
def calc_transac_price(row):
    if(row.event == "TokenPurchase"):
        price = abs(row.eth_delta/row.token_delta)
        return price
    elif(row.event == "EthPurchase"):
        price = abs(row.token_delta/row.eth_delta)
        return price

In [39]:
second_method = uniswap_history.copy()

In [51]:
second_method['before_P'] = second_method.apply(lambda row: uniswap_price_before_transaction(row), axis=1)
second_method['after_P'] = second_method.apply(lambda row: uniswap_price_after_transaction(row), axis=1)
second_method['unity_P'] = second_method.apply(lambda row: calc_transac_price(row), axis=1)
second_method['profit'] = second_method.apply(lambda row: calc_abs_profit(row.after_P, row), axis=1)
second_method.dropna(inplace=True)

In [55]:
second_method['event'].value_counts()

TokenPurchase    69193
EthPurchase      66416
Name: event, dtype: int64

In [57]:
second_method.loc[
#     (second_method['unity_P'] > second_method['after_P']) &
    (second_method['profit'] < 0.0)
]['event'].value_counts()

TokenPurchase    55742
EthPurchase      53567
Name: event, dtype: int64

## Terms
- before_P = price before transaction (using the balance before the transaction occurs);
- unity_P = price of transaction (using the deltas to calculate the unitary price of specific coin during transaction);
- after_P = price after transaction (using the balances plus deltas to calculate the price after a transaction occurs);


For an arbitrage agent, we must have after_P > unity_P so he can have profit in his transaction.

In [ ]:
def calc_before_P(row):
    row = uniswap_history.iloc[row.name-1] 
    if(row.event == 'TokenPurchase'):        
        spot_price = row.eth_balance / row.token_balance
        return Decimal(spot_price)
    elif(row.event == 'EthPurchase'):
        spot_price = row.token_balance / row.eth_balance
        return Decimal(spot_price)
    
def calc_unity_P(row):
    if(row.event == "TokenPurchase"):
        price = abs(row.eth_delta/row.token_delta)
        return price
    elif(row.event == "EthPurchase"):
        price = abs(row.token_delta/row.eth_delta)
        return price
    
def calc_after_P(row):
    if(row.event == 'TokenPurchase'):
        spot_price = row.eth_balance / row.token_balance
        return Decimal(spot_price)
    elif(row.event == 'EthPurchase'):
        spot_price = row.token_balance / row.eth_balance
        return Decimal(spot_price)
    
def calc_gross_profit(P, row):
    if(row.event == "TokenPurchase"):
        profit = int(abs(row.token_delta*P)) - row.eth_delta
        return profit
    elif(row.event == "EthPurchase"):
        profit = abs(row.eth_delta) - int(row.token_delta/P)
        return profit